# Hazelcast Python Client SQL Support Notebook

This notebook demonstrates the SQL support of Hazelcast Python Client. Hazelcast provides in-depth SQL support for Map data structures kept in Hazelcast Clusters. Using Hazelcast SQL support, you can create mappings between your data and a database table and execute SQL queries on the Map. This support provides fast in-memory computing using SQL without writing complex functions that iterate through your maps.</br></br>
Through this tutorial, you can either use your local cluster or Hazelcast Cloud. We will use Hazelcast Cloud as our cluster provider to not worry about setup or installation. Hazelcast Cloud offers free registration with a 200MB storage area. You can register using the following link and create a cluster in a few minutes: https://cloud.hazelcast.com/. </br></br> Remember that you can run this notebook in a Google Colabrotary environment without dealing with local installations. If you have any issues, you can ask your questions in Hazelcast Community channels, find links here: https://hazelcast.com/dev-community/ 

![Tux, the Linux mascot](https://media-exp1.licdn.com/dms/image/C561BAQEDzMqseo-qiA/company-background_10000/0/1626235318986?e=2147483647&v=beta&t=9QQp_S3AuK8-1nMqsSTlL0xy2_JpkNQrzFJ7D_SLUYU)


After creating a cluster at Hazelcast Cloud, we need your cluster name and discovery token to connect from this notebook. You can find this information from **Connect to your application > Advanced Setup** tab in the Hazelcast Cloud website.

## Hazelcast Cloud Authentication Tokens

In [104]:
CLUSTER_NAME = "YOUR_CLUSTER_NAME"
DISCOVERY_TOKEN = "YOUR_DISCOVERY_TOKEN"

## Connect to API

Instead of hard-coding data entries to our Map, we will use an API source to pull thousands of data to our cluster. We decided to use a cinema API named **The Movie Database (TMDB)** which returns in-depth movie and actor data for us as JSON objects. To be able to send a request to this API, you will need an API key. You can create it easily from their website. Note that you need to create an account first and then go to **Settings > API > Create New Key**. It may ask for some information about your project; providing short descriptions like "Experimenting API requests" is enough.

https://www.themoviedb.org/signup

In [5]:
API_KEY = "7f057be532c35bfdcb322d16c5cc406d"

Now, we need our Python client to use Hazelcast. You can embed Hazelcast inside your program by simply inserting a package, `hazelcast-python-client`, and utilize all the functionality using this package.

# Install Hazelcast Client

In [106]:
%pip install hazelcast-python-client

/Users/hazelcast/.zshenv:1: permission denied: /Users/hazelcast/Desktop
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Imports

In [1]:
import hazelcast
import json
import requests

# Connect To Hazelcast Cloud

We are ready to connect our cluster from Python program. Hazelcast uses config settings to find your cluster. We are providing our Cloud tokens for configuration to connect Hazelcast Cloud. After seeing the `Connection successful.` message, we can get our `Map` data structures using `get_map()` functions.

*Note: If you want to connect to a local cluster in Jupyter Notebook, you should remove the config options inside the `HazelcastClient()` method. In this case, it tries to connect`localhost:5701`*

In [107]:
client = hazelcast.HazelcastClient(
    cluster_name=CLUSTER_NAME,
    cloud_discovery_token=DISCOVERY_TOKEN,
    statistics_enabled=True
)

print("Connection successful")

Connection successful


We will need four different Maps for our scenario. We can them create using `get_map()` function.

*   movies: id, name, vote count, vote average, release date
*   casts: movie id, actor id pairs
*   actors: actor id, name, popularity, gender
*   reviews: author username, movie id, review content, rating 



In [3]:
movies  = client.get_map("movies").blocking()
casts   = client.get_map("casts").blocking()
reviews = client.get_map("reviews").blocking()
actors  = client.get_map("actors").blocking()

# Load Data from API

Now, we are ready to pull our data from the TMDB API and load them to our Hazelcast Cluster. To do that, we need to send requests to our API provider. Following cells send API requests and insert results into appropriate maps.

**This step may take time, depending on your connection speed and server latency. As long as there is no error and an `[*]` is present near to cell, do not worry.**

In [40]:
key = 0
for i in range(1,200):
    data = requests.get("https://api.themoviedb.org/3/movie/top_rated?api_key=%s&language=en-US&page=%d" % (API_KEY, i)).json()
    for movie in data['results']:
        movies.put(str(key), hazelcast.core.HazelcastJsonValue(movie))
        key += 1
print("Movie data has been loaded.")

Movie data has been loaded.


In [48]:
key = 0; cast_key = 0;
for i in range(1,100):
    data = requests.get("https://api.themoviedb.org/3/person/popular?api_key=%s&language=en-US&page=%d" % (API_KEY,i)).json()
    for a in data['results']:
        actors.put(str(key), hazelcast.core.HazelcastJsonValue(a))
        key += 1
        
        for c in a['known_for']:
            if c['media_type'] == 'movie':
                casts.put(str(cast_key), hazelcast.core.HazelcastJsonValue({"movie_id":c['id'], "actor_id":a['id']}))
                cast_key += 1
print("Data has been loaded.")

Data has been loaded.


In [49]:
key = 0
for i in range(1,200):
    value = movies.get(str(i)).to_string()
    movie = json.loads(value)
    url = "https://api.themoviedb.org/3/movie/%d/reviews?api_key=%s&language=en-US&page=1" % ( movie['id'], API_KEY)
    data = requests.get(url).json()
    for r in data['results']:
        review = {'movie_id':movie['id'], 'author_username': r['author_details']['username'], 'rating':r['author_details']['rating'], 'content':r['content']}
        reviews.put(str(key), hazelcast.core.HazelcastJsonValue(review))
        key += 1
print("Review data has been loaded.")

Review data has been loaded.


After the loading data operations, there should be approximately </br>
*   1K entry in `actors` map
*   4K entry in `casts` map
*   4K entry in `movies` map
*   500 entry in `reviews` map</br>

You can run following cell to control the Map sizes.

In [50]:
print(f"actors map size: {actors.size()}")
print(f"casts map size: {casts.size()}")
print(f"movies map size: {movies.size()}")
print(f"reviews map size: {reviews.size()}")

actors map size: 1980
casts map size: 4147
movies map size: 3980
reviews map size: 478


# Create Mapping between Map and Table

Here is the critical part. We inserted many  `HazelcastJsonValue()` objects into our Map. Now, we will create a mapping between our Map value fields and table columns. Using this mapping, we will execute SQL queries on our maps. You can select any column you want to map; not all fields are mandatory. Also, remember that our current value format must be `json-flat` to refer JSON fields directly in the query.

In [51]:
query = """
    CREATE OR REPLACE MAPPING movies (
        __key VARCHAR, 
        adult BOOLEAN,
        backdrop_path VARCHAR,
        id INT,
        original_value VARCHAR,
        original_title VARCHAR,
        overview VARCHAR,
        popularity DECIMAL,
        poster_path VARCHAR,
        release_date VARCHAR,
        title VARCHAR,
        video BOOLEAN,
        vote_average DECIMAL,
        vote_count INT
    )
    TYPE IMap OPTIONS(
        'keyFormat' = 'varchar',
        'valueFormat' = 'json-flat'
    )
"""
client.sql.execute(query).result()
print("Mapping for movies has been created.")

Mapping for movies has been created.


In [52]:
query = """
    CREATE OR REPLACE MAPPING casts (
        __key VARCHAR,
        movie_id INT,
        actor_id INT
    )
    TYPE IMap OPTIONS(
        'keyFormat' = 'varchar',
        'valueFormat' = 'json-flat'
    )
"""
client.sql.execute(query).result()
print("Mapping for cast has been created.")

Mapping for cast has been created.


In [53]:
query = """
    CREATE OR REPLACE MAPPING actors (
        __key VARCHAR,
        adult BOOLEAN,
        gender INT,
        id INT,
        known_for_department VARCHAR,
        name VARCHAR,
        popularity DECIMAL,
        profile_path VARCHAR
    )
    TYPE IMap OPTIONS(
        'keyFormat' = 'varchar',
        'valueFormat' = 'json-flat'
    )
"""
client.sql.execute(query).result()
print("Mapping for actors has been created.")

Mapping for actors has been created.


In [54]:
query = """
    CREATE OR REPLACE MAPPING reviews (
        __key VARCHAR,
        movie_id INT,
        author_name VARCHAR,
        author_username VARCHAR,
        rating INT,
        content VARCHAR
    )
    TYPE IMap OPTIONS(
        'keyFormat' = 'varchar',
        'valueFormat' = 'json-flat'
    )
"""
client.sql.execute(query).result()
print("Mapping for reviews has been created.")

Mapping for reviews has been created.


# Fun part: SQL queries 

In the following examples, we will observe different SQL funtions supported by Hazelcast and execute them to process our data quickly. You can edit the queries directly and experiment SQL support hands-on.  Fasten the belts!

## Movies released before 2015 and high vote average with cote count > 20K 

In this first query, we will use essential features of SQL, like the `WHERE` clause. We have some filters to select movies among 4K entries, like `vote_count`, `vote_average`, and `release_date`. You can use `AS` keyword to specify the column name of result. Without Hazelcast, typically, we need to iterate over our map and check whether the current entry fits into our definition or not. Also, this approach requires an additional structure to store our selected entries. With the inclusive SQL support of Hazelcast, we can translate our filters to a SQL query and execute it on our map. 

In [109]:
query = """
    SELECT m.title AS name
    FROM movies m
    WHERE m.vote_count > 20000 AND m.vote_average > 7 AND m.release_date < '2015-01-01'
    ORDER BY m.popularity DESC
"""

result = client.sql.execute(query).result()

for row in result:
    print(row['name'])

Avatar
Harry Potter and the Philosopher's Stone
The Avengers
Titanic
The Wolf of Wall Street
Interstellar
The Lord of the Rings: The Fellowship of the Ring
Iron Man
Inception
Shutter Island
The Shawshank Redemption
The Matrix
The Dark Knight
Forrest Gump
Guardians of the Galaxy
Fight Club
Pulp Fiction
Django Unchained


## Here all-star cast movies for you

Sometimes you may want to watch a movie with many star actors! Easy to find those movies among thousands of entries! You need to `JOIN` your two maps on `movie id` and filter those with a total actor popularity point larger than 500. You can use `SUM()` function to find the total value of a column for a group, in this case, for every movie since we are grouping according to `title` of movie. You don't have to deal with any loops, control statements, or additional storage!</br>

Spoiler alert! There are many super hero movie 🚀

In [108]:
query = """
    SELECT m.original_title as movieName
    FROM movies m
    JOIN casts c ON m.id = c.movie_id
    JOIN actors a ON c.actor_id = a.id
    GROUP BY m.original_title
    HAVING SUM(a.popularity) > 400
"""

result = client.sql.execute(query).result()

for row in result:
        print(row['movieName'])

Captain America: Civil War
Deadpool 2
Blade Runner 2049
The Dark Knight
Guardians of the Galaxy
Once Upon a Time… in Hollywood
Star Wars: The Force Awakens
Avengers: Endgame
The Avengers
The Dark Knight Rises
Fantastic Beasts and Where to Find Them
Avengers: Age of Ultron
Avengers: Infinity War
Inception
Thor: Ragnarok


## Get all the actors of a movie serie

In our map, we have series that have more than one movie and we have registered actors for each of these movies. Some of the actors are registered for multiple of them, like Daniel Radcliffe and Emma Watson. So, we will use `DISTINCT()` function to select each of them only once and get all the registered actor list. The tricky part in here is joining all the three maps and get the registered movies that contain `Harry Potter` inside name. If we try to do in plain code, you need to select whether the current actor is registered for a movie name contain `Harry Potter` and then re-control whether it has been added before, nearly `O(n^3)` complexity!

In [102]:
query = """
    SELECT DISTINCT(a.name), a.popularity
    FROM movies m
    JOIN casts c ON m.id = c.movie_id
    JOIN actors a ON c.actor_id = a.id
    WHERE m.original_title LIKE '%Harry Potter%'
    ORDER BY a.popularity DESC
"""

result = client.sql.execute(query).result()

for row in result:
        print("%s %.2f" % (row['name'], row['popularity']))

Ralph Fiennes 52.65
David Thewlis 50.89
Robert Pattinson 46.79
Emma Watson 43.43
Helena Bonham Carter 42.64
Jamie Campbell Bower 39.72
Gary Oldman 38.59
David Tennant 37.05
Fiona Shaw 35.79
Bill Nighy 35.51
J.K. Rowling 34.76
Daniel Radcliffe 32.29
Helen McCrory 31.38
Kenneth Branagh 30.93
Ciarán Hinds 29.86
Brendan Gleeson 29.39
Rhys Ifans 28.78
Domhnall Gleeson 28.30
Natalia Tena 28.20
Hero Fiennes Tiffin 27.43
Emma Thompson 26.33
Julianne Hough 25.64
Harry Melling 24.76
Jason Isaacs 24.45
Frank Dillane 24.31
Michael Gambon 24.19
Pam Ferris 23.96
David Yates 23.67
Shirley Henderson 22.86
Elarica Johnson 22.52
Michelle Fairley 18.86
Peter Mullan 15.76


## Get the movies that get the highest review ratings from users

Now, we will try to get the top ten movies with average ratings higher than eight and at least reviewed by five different users. Note that `reviews` are separate map and different than `vote` which is a column of `movies` table. It looks complicated, right? Typically, you must traverse the two maps in order square. First, get all the reviews for a movie, calculate the average rating, store results in another data structure, and repeat it for all the movies. In the end, you must sort the results in descending order and get the highest ten movies.

Don't give up, we are there for you! Using the Hazelcast SQL support, you can use the power of SQL functions and aggregate operations like `ORDER BY()`, `LIMIT()`, and `AVG()` on your Map! You need to write your SQL query as a string and then execute it on your map with just one line, `sql.Execute()`. It's that easy! This operation returns a table, which consists of rows. You can traverse inside results by a simple `for` loop.

In [110]:
query = """
    SELECT m.original_title as name, AVG(r.rating) as rating
    FROM movies m
    JOIN reviews r ON r.movie_id = m.id
    GROUP BY m.original_title
    HAVING AVG(r.rating) > 8 AND COUNT(*) > 5
    ORDER BY rating DESC
    LIMIT 10
"""

result = client.sql.execute(query).result()

for row in result:
        print("Movie Name: %s - Average Rating: %.2f" % (row['name'], row.get_object('rating')))

Movie Name: The Shawshank Redemption - Average Rating: 9.00
Movie Name: Django Unchained - Average Rating: 9.00
Movie Name: The Shining - Average Rating: 9.00
Movie Name: 기생충 - Average Rating: 8.86
Movie Name: The Lord of the Rings: The Fellowship of the Ring - Average Rating: 8.86
Movie Name: Il buono, il brutto, il cattivo - Average Rating: 8.67
Movie Name: Soul - Average Rating: 8.50
Movie Name: Joker - Average Rating: 8.50
Movie Name: Interstellar - Average Rating: 8.45
Movie Name: Green Book - Average Rating: 8.40


## Curious about user reviews?

There are many detailed reviews in our database. You can find a randomly selected entry for a movie or actor you are curious about. You can use the `LIKE` keyword for searching a specific keyword in our Map. For example, we searched for entries about Leanardo DiCaprio and got one between them randomly using the `RAND()` function. You can increase `LIMIT` to get more reviews and add multiple keywords. 

You can try the same thing with movie descriptions to find a specific movie informations, just use `movie.description` column!

In [64]:
query = """
    SELECT m.original_title, r.author_username, r.rating, r.content
    FROM movies m, reviews r
    WHERE r.movie_id=m.id AND 
          r.content LIKE '%Avengers%' AND 
          r.rating IS NOT NULL
    ORDER BY RAND()
    LIMIT 1
"""

result = client.sql.execute(query).result()

for row in result:
        print("Movie name: %s" % row.get_object('original_title'))
        print("-"*60)
        print("Review by %s rating is %d/10" % (row.get_object('author_username'), row.get_object('rating')))
        print("-"*60)
        print(row.get_object('content'))

Movie name: Avengers: Endgame
------------------------------------------------------------
Review by Ruuz rating is 9/10
------------------------------------------------------------
Hey, so, just some really quick thoughts I wanna get down, 'cause it's after 1 o'clock in the morning, and I wanna get some sleep so I can wake up well rested to see _Endgame_ again.

Putting it out there right from the get-go, _Avengers: Endgame_ is my favourite MCU movie. It's my favourite movie of the year too, but I've only seen like ten or so 2019 releases, and this is the 22nd Marvel Cinematic Universe film, so that seems far and away the bigger deal. I can't say it's perfect, there's some conflicting stuff within its own runtime that really doesn't seem to gel (though a re-watch may prove me wrong on that), and it's not like every moment was the no-holds-barred-zero-exception best version of that moment from start to end. But God I loved this thing.

I will say this though, the reason that I loved it

# Conculusion

These days the world is data heaven, and we have millions of data. Most of them are wasted due to the high workload of processing them. As you can see from the examples, even if it is a simple map search, the algorithmic complexity and effort to execute the filter are tremendous. Hazelcast offers a simple solution to evaluate and process all your data more easily and quickly as a real-time data processing platform. Using Hazelcast's SQL engine, you can skip all of the details and directly work on the value of your customer's data. You can infer much information without dealing with hundreds of line codes and slow executions.

You can follow our website and participate in many educational events. Also, we would love to answer your questions over our Community Slack Channel: